**The code below to create the 4x4 mapping should create a positive mapping**

In [1]:
# This code was created by Darshini Rajamani of Purdue University

import random;

def get_random_number():
    return random.randint(-9, 9)

def is_valid(matrix, rows, cols, row, col, num):
    # initial assigment to check validation
    matrix[row][col] = num

    # if row >= 2 and col == cols - 1:  # checking validatation atleast after filling two rows
    #     for i in range(cols):
    #         # check if x + w < 0
    #         if matrix[0][i] is not None and matrix[3][i] is not None:
    #             if matrix[0][i] + matrix[3][i] < 0:
    #                 matrix[row][col] = None  # Undo the initial assignment
    #                 return False

    #         # check if y + z < 0
    #         if matrix[1][i] is not None and matrix[2][i] is not None:
    #             if matrix[1][i] + matrix[2][i] < 0:
    #                 matrix[row][col] = None  # Undo the initial assignment
    #                 return False

    # Check row condition (check in E(2,2))
    if col == 3:
        if sum(matrix[row][:2]) != matrix[row][2] + num:
            return False

    # Check column condition (linearity check)
    if row == 3:
        if matrix[0][col] + matrix[1][col] != num + matrix[2][col]:
            return False

    # Positivity check
    if col == 3:
        matrix[row][col] = num
        if matrix[row][0] + matrix[row][2] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][0] + matrix[row][3] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][1] + matrix[row][2] < 0:
            matrix[row][col] = None
            return False
        if matrix[row][1] + matrix[row][3] < 0:
            matrix[row][col] = None
            return False

    # # Check diagonal condition
    # if row == 3 and col == 3:
    #     if matrix[0][3] + matrix[3][0] != matrix[1][2] + matrix[2][1]:
    #         return False

    # # Check for row uniqueness
    # # Note: this is needed because the rows would repeat without it
    # if col == 3:
    #     matrix[row][col] = num
    #     if tuple(matrix[row]) in set(map(tuple, matrix[:row])):
    #         matrix[row][col] = None
    #         return False
    #     matrix[row][col] = None

    # # Check that only one digit can be 0 in a row (to prevent 0,0,0,0) in a row)
    # if col == 3:
    #   zero_count = sum([1 for x in matrix[row] if x == 0])
    #   if zero_count > 1:
    #     return False
    # if all(x is not None for row in matrix for x in row):
    #     for i in range(cols):
    #         if matrix[1][i] + matrix[2][i] != matrix[0][i] + matrix[3][i]:
    #             return False

    matrix[row][col] = None
    return True

def create_matrix(rows, cols):
    matrix = [[None for _ in range(cols)] for _ in range(rows)]

    backtrackFailedAttempts = 0
    def backtrack(row, col):  # to check for complete and valid solution
        nonlocal backtrackFailedAttempts
        if backtrackFailedAttempts > 1000:
            return False

        if row == rows:
            return True

        nums = [i for i in range(-9, 10)]  # range [-9, 10)
        random.shuffle(nums)

        for num in nums:
            if is_valid(matrix, rows, cols, row, col, num):
                matrix[row][col] = num

                next_row = row
                next_col = col + 1
                if next_col == cols:
                    next_row += 1
                    next_col = 0

                if backtrack(next_row, next_col):
                    return True

        matrix[row][col] = None
        backtrackFailedAttempts = backtrackFailedAttempts + 1
        return False

    backtrack(0, 0)  # recursion

    return matrix


In [19]:
# this code was created by Luke Luschwitz and Karim with very heavy editing by Luke. Abbas and Darshini added some code and influence as well

from scipy.optimize import linprog
import numpy as np

numberOfMappingsToCreate = 4000
#######################
### Create Mappings ###
#######################
listOfMappings = []
for i in range(numberOfMappingsToCreate):
  rows = 4
  cols = 4

  isAValidMatrix = False

  while not isAValidMatrix:
    isAValidMatrix = True
    matrix = create_matrix(rows, cols)
    for row in matrix:
      for element in row:
        if element is None:
          isAValidMatrix = False

  # matrix[1], matrix[3] = matrix[3], matrix[1]

  listOfMappings.append(matrix)


###########################
# Check for Extendability #
###########################
extendableMappings = []
nonExtendableMappings = []

for matrix in listOfMappings:
  mat = [1, 1, 1, 1]
  for_A_ub = [[1,0,0,0],
            [0,1,0,0],
            [0,0,1,0],
            [0,0,0,1],
            [-1,0,0,0],
            [0,-1,0,0],
            [0,0,-1,0],
            [0,0,0,-1],]
  solve_for = [[min(matrix[0][2], matrix[0][3])],
             [min(matrix[1][2], matrix[1][3])],
             [min(matrix[2][2], matrix[2][3])],
             [min(matrix[3][2], matrix[3][3])],
             [-max(-matrix[0][0], -matrix[0][1])],
             [-max(-matrix[1][0], -matrix[1][1])],
             [-max(-matrix[2][0], -matrix[2][1])],
             [-max(-matrix[3][0], -matrix[3][1])]]
  for_A_eq = [[1,1,-1,-1]]
  for_b_eq = [0]

  result = linprog(c=mat, A_ub = for_A_ub, b_ub = solve_for, A_eq = for_A_eq, b_eq = for_b_eq, bounds = None)
  for row in matrix:
    print(row)
  if (result.success):
    extendableMappings.append(matrix)
    print("Extendable")
  else:
    nonExtendableMappings.append(matrix)
    print("Not extendable")
  print()

# Save the sample data to a file
np.save('/content/extendableMappings.npy', extendableMappings)
np.save('/content/nonExtendableMappings.npy', nonExtendableMappings)

Streaming output truncated to the last 5000 lines.
Extendable

[4, 1, 6, -1]
[8, 9, 8, 9]
[5, 9, 5, 9]
[7, 1, 9, -1]
Not extendable

[7, 2, 8, 1]
[6, 9, 7, 8]
[5, 9, 8, 6]
[8, 2, 7, 3]
Extendable

[8, 2, 9, 1]
[6, 2, 4, 4]
[7, 4, 7, 4]
[7, 0, 6, 1]
Extendable

[4, 8, 6, 6]
[1, 2, 1, 2]
[5, 2, 2, 5]
[0, 8, 5, 3]
Extendable

[-1, 6, 2, 3]
[9, 4, 5, 8]
[0, 4, 0, 4]
[8, 6, 7, 7]
Extendable

[5, -1, 2, 2]
[9, 6, 6, 9]
[9, 0, 6, 3]
[5, 5, 2, 8]
Extendable

[-1, 7, 4, 2]
[4, 3, 6, 1]
[-3, 9, 3, 3]
[6, 1, 7, 0]
Extendable

[5, 4, 1, 8]
[-2, 7, 2, 3]
[2, 9, 4, 7]
[1, 2, -1, 4]
Not extendable

[0, 1, 0, 1]
[3, 2, 0, 5]
[3, 2, -1, 6]
[0, 1, 1, 0]
Not extendable

[4, -1, 2, 1]
[-1, 4, 1, 2]
[4, 0, 2, 2]
[-1, 3, 1, 1]
Extendable

[3, -1, 1, 1]
[3, 9, 6, 6]
[2, 3, 4, 1]
[4, 5, 3, 6]
Extendable

[9, 4, 7, 6]
[-1, 4, 1, 2]
[2, 9, 5, 6]
[6, -1, 3, 2]
Extendable

[7, 9, 9, 7]
[6, 9, 8, 7]
[4, 9, 8, 5]
[9, 9, 9, 9]
Extendable

[7, 8, 8, 7]
[9, 8, 8, 9]
[9, 8, 8, 9]
[7, 8, 8, 7]
Extendable

[9, 4, 7, 6]
[

In [ ]:
import numpy as np
extendableMappings = np.load('/content/extendableMappings.npy')
nonExtendableMappings = np.load('/content/nonExtendableMappings.npy')

In [20]:
print(f"extendableMappings: {len(extendableMappings)}")
print(f"nonExtendableMappings: {len(nonExtendableMappings)}")

extendableMappings: 3119
nonExtendableMappings: 881


In [ ]:
# This code was created by Luke and Karim

from scipy.optimize import linprog

# The points in P(2,2) that were mapped from E(2,2).
# Points are mapped by f: E(2,2) -> P(2,2)

# These are what x, y, z, and w represent
# x = [a1,b1,c1,d1]
# y = [a2,b2,c2,d2]
# z = [a3,b3,c3,d3]
# w = [a4,b4,c4,d4]

# This matrix we know is extendible
x = [-1,3,1,1] # x is a mapped point in P(2,2)
y = [0,1,1,0] # y is a mapped point in P(2,2)
z = [0,2,1,1] # z is a mapped point in P(2,2)
w = [1,0,1,0] # w is a mapped point in P(2,2)

# The matrix that we are trying to map to R4
matrix = [x,
          y,
          z,
          w]

# In the scipy.optimize.linprog documentation, mat is c: "The coefficients of the linear objective function to be minimized".
# On Prof. SinClair's blackboard, mat is t: the vector of variables t1,t2,t3,t4 that need to be minimized/maximized. In our case, the objective function does not matter, we only care whether the solution exists.
mat = [1, 1, 1, 1]

# for_A_ub represents the system of inequalities
for_A_ub = [[1,0,0,0],
            [0,1,0,0],
            [0,0,1,0],
            [0,0,0,1],
            [-1,0,0,0],
            [0,-1,0,0],
            [0,0,-1,0],
            [0,0,0,-1],]

# In the scipy.optimize.linprog documentation, solve_for is b_ub: "The inequality constraint vector. Each element represents an upper bound on the corresponding value of A_ub @ x."
# On Prof. SinClair's blackboard, this is r: [r1,r2,r3,r4,-l1,-l2,-l3,-l4].
solve_for = [[min(matrix[0][2],matrix[0][3])],
             [min(matrix[1][2], matrix[1][3])],
             [min(matrix[2][2], matrix[2][3])],
             [min(matrix[3][2], matrix[3][3])],
             [-max(-matrix[0][0], -matrix[0][1])],
             [-max(-matrix[1][0], -matrix[1][1])],
             [-max(-matrix[2][0], -matrix[2][1])],
             [-max(-matrix[3][0], -matrix[3][1])]]

# In the scipy.optimize.linprog documentation, for_A_eq is A_eq: "The equality constraint matrix. Each row of A_eq specifies the coefficients of a linear equality constraint on x."
# On Prof. SinClair's blackboard, this is t1-t2-t3+t4=0.
for_A_eq = [[1,1,-1,-1]]

# In the scipy.optimize.linprog documentation, for_b_eq is b_eq: "The equality constraint vector. Each element of A_eq @ x must equal the corresponding element of b_eq."
# On Prof. SinClair's blackboard, this is t1-t2-t3+t4=0.
for_b_eq = [0]


# Solve for inequalities with scipy.optimize.linprog
result = linprog(c=mat, A_ub = for_A_ub, b_ub = solve_for, A_eq = for_A_eq, b_eq = for_b_eq, bounds = None)
# for how this function works, it has it such that A_ub <= b_ub at whatever c is equal to


In [21]:
# All the code below was made by Luke with slight influence from Abbas

import numpy as np

# This function returns the matrix B that is farthest away from the matrices in setA
def farthestBFromA(setB, setA):
  totalDistancesFromA = []
  # iterate through each B matrix
  for B in setB:
    currentBTotalDistance = 0
    # accumulate sum of distances from A to B (distance squared)
    for A in setA:
      currentBTotalDistance = currentBTotalDistance + (np.linalg.norm(np.subtract(B, A)) ** 2)
    totalDistancesFromA.append(currentBTotalDistance)

  orderedBsByDistance = []
  for i in range(len(totalDistancesFromA)):
    index = np.argmax(totalDistancesFromA)
    orderedBsByDistance.append(setB[index])
    totalDistancesFromA[index] = -np.inf
  return orderedBsByDistance



farthestBs = farthestBFromA(nonExtendableMappings, extendableMappings)

for i in range(len(farthestBs[0:3])):
  for row in farthestBs[i]:
      print(row)
  print(f"{i}th farthest from the extendables")
  print()

[9, -3, 3, 3]
[7, 1, 9, -1]
[8, -2, 4, 2]
[8, 0, 8, 0]
0th farthest from the extendables

[9, 0, 9, 0]
[6, -1, 4, 1]
[9, -2, 5, 2]
[6, 1, 8, -1]
1th farthest from the extendables

[9, 0, 8, 1]
[2, 2, 6, -2]
[2, 3, 7, -2]
[9, -1, 7, 1]
2th farthest from the extendables



In [77]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score

# Ensure everything is a numpy array
extendableMappings = np.array(extendableMappings)
nonExtendableMappings = np.array(nonExtendableMappings)
farthestBs = np.array(farthestBs)

# Generate input by adding the farthestB matrix to the list of extendableMappings
features = np.concatenate((
    extendableMappings,
    [farthestBs[0]],
    [extendableMappings[0]/1000],
    [nonExtendableMappings[0]/1000]
    ))
print(f"len(features): {len(features)}")

# Flatten the matrices into 1D arrays
features = features.reshape(len(features), -1)

# Generate labels (0 or 1)
labels = [0]*len(extendableMappings) + [1,0,1]
print(f"len(labels): {len(labels)}")
print(f"labels: {labels}")

# Create and train a logistic regression model
model = svm.SVC(kernel='linear', C=1e10, coef0=0.0, tol=1e-5)
model.fit(features, labels)

# Set the bias (intercept) to be zero
model.intercept_ = [0.0]

# Make predictions on the test set
predictions = model.predict(features)

# Evaluate the accuracy of the model
accuracy = accuracy_score(labels, predictions)
print(f"Accuracy: {accuracy}")

# Get the coefficients (weights) of the hyperplane
# coefficients = model.coef_.reshape(4,4)
coefficients = model.coef_

# Intercept of the hyperplane
intercept = model.intercept_

print("Coefficients:\n", coefficients) #the matrix that we get from the ML program
print("Intercept:", intercept)


len(features): 3122
len(labels): 3122
labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [78]:
print("coefficients")
print(coefficients.reshape(4,-1))

classifier = np.round(coefficients*2) # this is coefficients, but it is doubled and rounded to integers
print("classifier")
print(classifier.reshape(4,-1))

print("testClassifier") # this is the same as classifier, but fiddled to have zeros and small integers
testClassifier = np.array([0, 100, -20, 100,
                           0, -300, 30, -300,
                           -20, -130, 0, -150,
                           0, -120, 0, -130])
testClassifier = testClassifier / 10
print(testClassifier.reshape(4,-1))
print()

allMappings = np.concatenate((extendableMappings, nonExtendableMappings), axis=0)
allMappings = allMappings.reshape(len(allMappings), -1)
print()

totalMappingsInClass1 = 0
totalMappingsInClass2 = 0
classifiedMappingResults = []

for i in range(len(allMappings)):
  dotProduct = np.dot(testClassifier, allMappings[i])
  dotProduct = np.sum(dotProduct)
  extendableOrNot = "E" if i<len(extendableMappings) else "N"
  if dotProduct < -0.99061298:
    # print(f"{extendableOrNot} {dotProduct} (class 1)")
    classifiedMappingResults.append((allMappings[i], extendableOrNot, 1))
    totalMappingsInClass1 += 1
  else:
    # print(f"{extendableOrNot} {dotProduct} (class 2)")
    classifiedMappingResults.append((allMappings[i], extendableOrNot, 2))
    totalMappingsInClass2 += 1

for mapping, extendability, _class in classifiedMappingResults:
  if (extendability == "E" and _class == 2):
    print("Bad classifier, extendableMapping in class 2")
    # print(mapping, extendability, _class)
    break
  if totalMappingsInClass1 < 1:
    print("Bad classifier, no mappings in class 1")
    break
  if totalMappingsInClass2 < 1:
    print("Bad classifier, no mappings in class 2")
    break

print(f"totalMappingsInClass1 {totalMappingsInClass1}")
print(f"totalMappingsInClass2 {totalMappingsInClass2}")


coefficients
[[ -11.09147285   49.56891696   -9.46142806   47.93887217]
 [   1.94360246 -177.77352401   16.73056701 -192.56048857]
 [  -8.74831119  -66.8002957     2.33881337  -77.88742025]
 [  -0.39955921  -61.40431135    4.93032559  -66.73419614]]
classifier
[[ -22.   99.  -19.   96.]
 [   4. -356.   33. -385.]
 [ -17. -134.    5. -156.]
 [  -1. -123.   10. -133.]]
testClassifier
[[  0.  10.  -2.  10.]
 [  0. -30.   3. -30.]
 [ -2. -13.   0. -15.]
 [  0. -12.   0. -13.]]


totalMappingsInClass1 3996
totalMappingsInClass2 4


In [ ]:
import numpy as np

test = np.array([[0,1,2,3],
                 [4,5,6,7],
                 [8,9,10,11],
                 [12,13,14,15]])

test = np.array([test, test])

print(test)

print(test.reshape(-1))

print(test.reshape(len(test), -1))

print(test.reshape(len(test), -1).reshape(len(test), -1))

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]]

 [[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]]]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15  0  1  2  3  4  5  6  7
  8  9 10 11 12 13 14 15]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
